In [0]:
import numpy as np
import random

from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras import backend as K

In [0]:
num_classes = 10
epochs = 20

In [0]:
def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [0]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [0]:

def contrastive_loss(y_true, y_pred):
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)

In [0]:
def create_pairs(x, digit_indices):
    '''
    Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [0]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

In [0]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
train_pairs, train_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
test_pairs, test_y = create_pairs(x_test, digit_indices)

In [10]:
train_pairs.shape, test_pairs.shape

((108400, 2, 28, 28), (17820, 2, 28, 28))

In [0]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

In [14]:
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          133504      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]                    
          

In [19]:
model.compile(loss=contrastive_loss, optimizer='rmsprop', metrics=[accuracy])
callback = EarlyStopping(monitor='val_loss', patience=2)

try:
    model.fit([train_pairs[:, 0], train_pairs[:, 1]], train_y, batch_size=128,
          epochs=epochs, validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_y), callbacks=[callback])
except(KeyboardInterrupt):
    print('\nStoped')

Train on 108400 samples, validate on 17820 samples
Epoch 1/20
108400/108400 [==============================] - 7s 61us/step - loss: 0.0948 - accuracy: 0.8878 - val_loss: 0.0441 - val_accuracy: 0.9553
Epoch 2/20
108400/108400 [==============================] - 6s 53us/step - loss: 0.0411 - accuracy: 0.9605 - val_loss: 0.0323 - val_accuracy: 0.9662
Epoch 3/20
108400/108400 [==============================] - 6s 53us/step - loss: 0.0292 - accuracy: 0.9716 - val_loss: 0.0272 - val_accuracy: 0.9719
Epoch 4/20
108400/108400 [==============================] - 6s 53us/step - loss: 0.0240 - accuracy: 0.9759 - val_loss: 0.0265 - val_accuracy: 0.9693
Epoch 5/20
108400/108400 [==============================] - 6s 54us/step - loss: 0.0205 - accuracy: 0.9797 - val_loss: 0.0252 - val_accuracy: 0.9715
Epoch 6/20
108400/108400 [==============================] - 6s 54us/step - loss: 0.0184 - accuracy: 0.9815 - val_loss: 0.0250 - val_accuracy: 0.9709
Epoch 7/20
108400/108400 [=============================

In [0]:
# compute final accuracy on training and test sets
y_pred = model.predict([train_pairs[:, 0], train_pairs[:, 1]])
train_acc = compute_accuracy(train_y, y_pred)
y_pred = model.predict([test_pairs[:, 0], test_pairs[:, 1]])
test_acc = compute_accuracy(test_y, y_pred)

In [24]:
print('* Accuracy on training set: %0.2f%%' % (100 * train_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * test_acc))

* Accuracy on training set: 99.32%
* Accuracy on test set: 97.29%
